# Make S2 Geomedian with different configurations

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pyproj import Transformer

import datacube

import sys
sys.path.append('../Scripts')
from deafrica_dask import create_local_dask_cluster
from s2_geomedian import run_tile


In [2]:
create_local_dask_cluster(n_workers=15)#,threads_per_worker=1)

Client Scheduler: tcp://127.0.0.1:36351 Dashboard: /user/fangfy/proxy/8787/status,Cluster Workers: 15 Cores: 15 Memory: 104.37 GB


## Generate seasonal geomedian for 2017 and 2019

In [3]:

def run_seasonal_gm(lat, lon, tilesize, cloud_buffer):
    
    # convert to projected centre coordinates
    transformer = Transformer.from_crs("epsg:4326", "epsg:6933", always_xy=True)
    x0, y0 = transformer.transform(lon, lat)

    x = (x0-tilesize/2, x0+tilesize/2)
    y = (y0-tilesize/2, y0+tilesize/2)

    times = [
        # quarter
        ("2017-01-01", "2017-03-31"),
        ("2017-04-01", "2017-06-30"),
        ("2017-07-01", "2017-09-30"),
        ("2017-10-01", "2017-12-31"),
        # 6 month
        ("2017-01-01", "2017-06-30"),
        ("2017-07-01", "2017-12-31"),
        # 12 month
        ("2017-01-01", "2017-12-31"),
        # quarter
        ("2019-01-01", "2019-03-31"),
        ("2019-04-01", "2019-06-30"),
        ("2019-07-01", "2019-09-30"),
        ("2019-10-01", "2019-12-31"),
        # 6 month
        ("2019-01-01", "2019-06-30"),
        ("2019-07-01", "2019-12-31"),
        # 12 month
        ("2019-01-01", "2019-12-31"),
        ]

    dask_chunks = {'time': -1, 'x':200, 'y': 200}

    filenames = []
    for time in times:
        if cloud_buffer>0:
            filename = "s2_geomedian/cloud_cover_0_100/s2_%s_%s_%d_%d_buffer%d.nc"%(time[0], time[1], x0, y0, cloud_buffer)
        else:
            filename = "s2_geomedian/cloud_cover_0_100/s2_%s_%s_%d_%d.nc"%(time[0], time[1], x0, y0)
        print(filename)
        filenames.append(filename)
    
        gm = run_tile(x, y, time, filename,
                  cloud_labels=[8,9,10], cloud_buffer=cloud_buffer, bad_labels=[0,1,3], 
                  bands=['blue','green','red','nir_2','swir_1'], # only testing a few bands for speed
                  redo=False, 
                  dask_chunks = dask_chunks, cloud_cover=[0, 100]
                 )


In [ ]:
print('Buruni ==============================================================================')
lat, lon = -3.1, 29.4
tilesize = 50000 # 50 km
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer)

lat, lon = -3.1, 29.4
tilesize = 50000 # 50 km
cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer)

print('Lagos, Nigeria ==============================================================================')
lat, lon = 6.6, 3.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer)

lat, lon = 6.6, 3.3
tilesize = 50000
cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer)

print('salt pan, Botswana ==============================================================================')
lat, lon = -20.7, 25.3
tilesize = 50000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer)

print('Congo/Gabon ==============================================================================')
lat, lon = -2.3, 12.8
tilesize = 100000
cloud_buffer=0
run_seasonal_gm(lat, lon, tilesize, cloud_buffer)

cloud_buffer=10
run_seasonal_gm(lat, lon, tilesize, cloud_buffer)


Buruni ==============================================================================
s2_geomedian/cloud_cover_0_100/s2_2017-01-01_2017-03-31_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2017-04-01_2017-06-30_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2017-07-01_2017-09-30_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2017-10-01_2017-12-31_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2017-01-01_2017-06-30_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2017-07-01_2017-12-31_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2017-01-01_2017-12-31_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2019-01-01_2019-03-31_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2019-04-01_2019-06-30_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2019-07-01_2019-09-30_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2019-10-01_2019-12-31_2836696_-395289.nc
s2_geomedian/cloud_cover_0_100/s2_2019-01-01_2019-06-30_2836696_-395289.nc
s2_geomedian/c